导入库

In [1]:
import win32com.client
import pythoncom
from pprint import pprint

xlDown = -4121
xlShiftDown = -4161

创建Application对象

In [2]:
excel_application = win32com.client.Dispatch("Excel.Application")  #create an excel application

使Excel可见

https://docs.microsoft.com/en-us/office/vba/api/excel.application.visible

In [3]:
excel_application.Visible = True

打开工作薄

https://docs.microsoft.com/en-us/office/vba/api/excel.workbooks.open

In [4]:
path = r"D:\test\python\test.xlsx"
workbooks = excel_application.Workbooks.Open(Filename=path)

打开某个sheet

https://docs.microsoft.com/en-us/office/vba/api/excel.workbook.worksheets

In [7]:
worksheets = workbooks.Worksheets

In [8]:
sheet_name = "Sheet1"  # 选定sheet的名字

In [9]:
original_worksheet = worksheets(sheet_name)

复制sheet

https://docs.microsoft.com/en-us/office/vba/api/excel.sheets.copy

In [12]:
original_worksheet.Copy(None, original_worksheet)   

In [13]:
new_worksheet = worksheets(original_worksheet.Index + 1)

In [14]:
new_worksheet.Name = "new_" + sheet_name    # 重命名sheet

In [15]:
from collections import OrderedDict

indent_level = 1
old_indent = 0
max_indent_level = 1
indent_dict = OrderedDict()
indent_level_dict = {"0" : 0}

通过Range对象的IndentLevel属性来判断是几级指标

https://docs.microsoft.com/en-us/office/vba/api/excel.range.indentlevel

由于缩进不统一，比如二级缩进有时是2，有时是3，但是每一组指标不同级别的缩进是统一的。所以首先利用一个字典，存储IndentLevel和其对应的指标级别，然后再利用第二个有序字典存储指标和其对应的指标级别。

In [16]:
for item in new_worksheet.Range("A2", new_worksheet.Range("A2").End(xlDown)):
    if item.IndentLevel > old_indent:
        indent_level = indent_level + 1
        old_indent = item.IndentLevel
        if max_indent_level < indent_level:
            max_indent_level = indent_level
    elif item.IndentLevel < old_indent:
        indent_level = indent_level_dict[str(item.IndentLevel)]
        old_indent = item.IndentLevel 
    indent_level_dict[str(item.IndentLevel)] = indent_level
    indent_dict[item.Value] = indent_level_dict[str(item.IndentLevel)]
print(max_indent_level)

3


通过Range的Insert方法插入列

https://docs.microsoft.com/en-us/office/vba/api/excel.range.insert

In [21]:
# 指标有几级就新建几列
for i in range(0, max_indent_level):
    new_worksheet.Range("A1", new_worksheet.Range("A1").End(xlDown)).Insert(Shift=xlShiftDown)
    new_worksheet.Cells(1, 1).Value = str(max_indent_level-i) + "级指标"

In [22]:
row = 2
row_content = {}
for item, indent_level in indent_dict.items():
    for column in range(indent_level, max_indent_level+1):
        row_content[str(column)] = item
    for column in range(1, max_indent_level+1):
        new_worksheet.Cells(row, column).Value = row_content[str(column)]
    row = row + 1    

In [65]:
workbooks.Save()    # 保存退出